In [ ]:
from google.cloud import aiplatform
import tensorflow as tf
from google.cloud import aiplatform
import base64

In [1]:
GOOGLE_CLOUD_PROJECT = 'brldi-gcpcapabilities-ai-audit'
GOOGLE_CLOUD_REGION = 'us-central1'
GCS_BUCKET_NAME = 'finalrundemo1chicago'
FILE_NAME ='Final_Chicago_Train.csv'

In [2]:


def register_model_and_manage_endpoint(
    project: str,
    location: str,
    model_display_name: str,
    model_artifact_uri: str,
    serving_container_image_uri: str,
    endpoint_display_name: str
):
    """Registers a trained model in Vertex AI Model Registry or notifies if it already exists,
    and manages an endpoint for the model.

    Args:
        project (str): Google Cloud project ID.
        location (str): Google Cloud region.
        model_display_name (str): Display name for the registered model.
        model_artifact_uri (str): GCS URI where the model artifacts are stored.
        serving_container_image_uri (str): Container image URI for serving the model.
        endpoint_display_name (str): Display name for the created endpoint.

    Returns:
        Dictionary with model ID and endpoint ID (numeric part only).
    """
    aiplatform.init(project=project, location=location)

    # Check if the model is already registered
    filter_expression = f"display_name='{model_display_name}'"
    models = aiplatform.Model.list(filter=filter_expression)
    if models:
        model_id = models[0].resource_name
        print(f"Model already registered with ID: {model_id}")
    else:
        model = aiplatform.Model.upload(
            display_name=model_display_name,
            artifact_uri=model_artifact_uri,
            serving_container_image_uri=serving_container_image_uri
        )
        model_id = model.resource_name
        print(f"Model registered successfully with ID: {model_id}")

    # Check if the endpoint already exists
    endpoints = aiplatform.Endpoint.list(filter=f"display_name='{endpoint_display_name}'")
    if endpoints:
        endpoint_id = endpoints[0].resource_name.split('/')[-1]  # Extracting the ID part
        print(f"Endpoint already exists with ID: {endpoint_id}")
    else:
        endpoint = aiplatform.Endpoint.create(
            display_name=endpoint_display_name,
            project=project,
            location=location
        )
        endpoint_id = endpoint.resource_name.split('/')[-1]  # Extracting the ID part
        print(f"Endpoint created successfully with ID: {endpoint_id}")

        # Deploy the model to the created endpoint
        endpoint.deploy(
            model=model,
            deployed_model_display_name=model_display_name,
            machine_type="n1-standard-4"
        )
        print(f"Model '{model_display_name}' deployed to endpoint '{endpoint_display_name}'.")

    return {"model_id": model_id, "endpoint_id": endpoint_id}
    return endpoint_id

# Set up the parameters for the model registration and endpoint creation
PROJECT_ID = GOOGLE_CLOUD_PROJECT
LOCATION = GOOGLE_CLOUD_REGION  # or other GCP region
MODEL_DISPLAY_NAME = GCS_BUCKET_NAME + '_auto_register'
MODEL_ARTIFACT_URI = 'gs://finalrundemo1chicago/chicago-gcp-vertex-pipelines/serving_model/1717051004/'
SERVING_CONTAINER_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-13:latest'
ENDPOINT_DISPLAY_NAME = 'demo1_auto_endpoint'

# Register the model and create endpoint
endpoint_name = register_model_and_manage_endpoint(
    project=PROJECT_ID,
    location=LOCATION,
    model_display_name=MODEL_DISPLAY_NAME,
    model_artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
    endpoint_display_name=ENDPOINT_DISPLAY_NAME
)




Creating Model
Create Model backing LRO: projects/971203737354/locations/us-central1/models/6296578736342958080/operations/5984551939268935680
Model created. Resource name: projects/971203737354/locations/us-central1/models/6296578736342958080@1
To use this Model in another session:
model = aiplatform.Model('projects/971203737354/locations/us-central1/models/6296578736342958080@1')
Model registered successfully with ID: projects/971203737354/locations/us-central1/models/6296578736342958080
Creating Endpoint
Create Endpoint backing LRO: projects/971203737354/locations/us-central1/endpoints/7933220485985206272/operations/8166545958729940992
Endpoint created. Resource name: projects/971203737354/locations/us-central1/endpoints/7933220485985206272
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/971203737354/locations/us-central1/endpoints/7933220485985206272')
Endpoint created successfully with ID: 7933220485985206272
Deploying Model projects/971203737354/

In [3]:
ENDPOINT = endpoint_name["endpoint_id"]
ENDPOINT

'7933220485985206272'

In [4]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):  # Handle TensorFlow tensors
        value = value.numpy()
    if isinstance(value, str):  # Convert string to bytes
        value = value.encode('utf-8')
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_taxi_trip_example(unique_key, taxi_id, trip_start_timestamp, trip_end_timestamp, trip_seconds,
                                trip_miles, pickup_census_tract, dropoff_census_tract, pickup_community_area,
                                dropoff_community_area, fare, tips, tolls, extras, trip_total, payment_type,
                                company, pickup_latitude, pickup_longitude, pickup_location,
                                dropoff_latitude, dropoff_longitude, dropoff_location, trip_start_day,
                                trip_start_month, trip_start_hour):
    """Creates a tf.train.Example message for a taxi trip to be sent to the model."""
    feature = {
        'unique_key': _bytes_feature(unique_key),
        'taxi_id': _bytes_feature(taxi_id),
        'trip_start_timestamp': _bytes_feature(trip_start_timestamp),
        'trip_end_timestamp': _bytes_feature(trip_end_timestamp),
        'trip_seconds': _int64_feature(trip_seconds),
        'trip_miles': _float_feature(trip_miles),
        'pickup_census_tract': _int64_feature(pickup_census_tract),
        'dropoff_census_tract': _int64_feature(dropoff_census_tract),
        'pickup_community_area': _int64_feature(pickup_community_area),
        'dropoff_community_area': _int64_feature(dropoff_community_area),
        'fare': _float_feature(fare),
        'tips': _float_feature(tips),
        'tolls': _float_feature(tolls),
        'extras': _float_feature(extras),
        'trip_total': _float_feature(trip_total),
        'payment_type': _bytes_feature(payment_type),
        'company': _bytes_feature(company),
        'pickup_latitude': _float_feature(pickup_latitude),
        'pickup_longitude': _float_feature(pickup_longitude),
        'pickup_location': _bytes_feature(pickup_location),
        'dropoff_latitude': _float_feature(dropoff_latitude),
        'dropoff_longitude': _float_feature(dropoff_longitude),
        'dropoff_location': _bytes_feature(dropoff_location),
        'trip_start_day': _int64_feature(trip_start_day),
        'trip_start_month': _int64_feature(trip_start_month),
        'trip_start_hour': _int64_feature(trip_start_hour)
    }

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()




2024-06-05 06:14:59.039711: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 06:15:00.623096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 06:15:00.623203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 06:15:00.901622: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-05 06:15:01.530829: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 06:15:01.537371: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
packet = {
    'unique_key': 'df8d9f937b137bd0',
    'taxi_id': 'Chicago_072',
    'trip_start_timestamp': '2023-05-01T07:30:00Z',
    'trip_end_timestamp': '2023-05-01T07:45:00Z',
    'trip_seconds': 900,
    'trip_miles': 2.5,
    'pickup_census_tract': 17031839100,
    'dropoff_census_tract': 17031980000,
    'pickup_community_area': 8,
    'dropoff_community_area': 24,
    'fare': 12.75,
    'tips': 3.00,
    'tolls': 0.00,
    'extras': 1.50,
    'trip_total': 17.25,
    'payment_type': 'Credit Card',
    'company': 'Taxi Affiliation Services',
    'pickup_latitude': 41.892073,
    'pickup_longitude': -87.628874,
    'pickup_location': 'POINT (-87.628874 41.892073)',
    'dropoff_latitude': 42.899602,
    'dropoff_longitude': -87.633308,
    'dropoff_location': 'POINT (-87.633308 41.899602)',
    'trip_start_day': 1,
    'trip_start_month': 5,
    'trip_start_hour': 7
}

serialized_example = serialize_taxi_trip_example(**packet)
print(f"Serialized_example: {serialized_example}")

Serialized_example: b'\n\xb7\x06\n\x10\n\x04fare\x12\x08\x12\x06\n\x04\x00\x00LA\n"\n\nunique_key\x12\x14\n\x12\n\x10df8d9f937b137bd0\n.\n\x12trip_end_timestamp\x12\x18\n\x16\n\x142023-05-01T07:45:00Z\n(\n\x07company\x12\x1d\n\x1b\n\x19Taxi Affiliation Services\n4\n\x10dropoff_location\x12 \n\x1e\n\x1cPOINT (-87.633308 41.899602)\n\x18\n\x0ftrip_start_hour\x12\x05\x1a\x03\n\x01\x07\n\x19\n\x10trip_start_month\x12\x05\x1a\x03\n\x01\x05\n \n\x13pickup_census_tract\x12\t\x1a\x07\n\x05\xfc\xfa\xb4\xb9?\n\x1f\n\x0cpayment_type\x12\x0f\n\r\n\x0bCredit Card\n\x1f\n\x16dropoff_community_area\x12\x05\x1a\x03\n\x01\x18\n\x1c\n\x10pickup_longitude\x12\x08\x12\x06\n\x04\xfcA\xaf\xc2\n\x10\n\x04tips\x12\x08\x12\x06\n\x04\x00\x00@@\n\x1b\n\x0fpickup_latitude\x12\x08\x12\x06\n\x04|\x91\'B\n\x1d\n\x11dropoff_longitude\x12\x08\x12\x06\n\x04AD\xaf\xc2\n\x1e\n\x15pickup_community_area\x12\x05\x1a\x03\n\x01\x08\n\x1c\n\x10dropoff_latitude\x12\x08\x12\x06\n\x041\x99+B\n\x16\n\ntrip_total\x12\x08\x12\x06\n\

In [6]:

example_proto = tf.train.Example.FromString(serialized_example)
example_proto

features {
  feature {
    key: "company"
    value {
      bytes_list {
        value: "Taxi Affiliation Services"
      }
    }
  }
  feature {
    key: "dropoff_census_tract"
    value {
      int64_list {
        value: 17031980000
      }
    }
  }
  feature {
    key: "dropoff_community_area"
    value {
      int64_list {
        value: 24
      }
    }
  }
  feature {
    key: "dropoff_latitude"
    value {
      float_list {
        value: 42.899600982666016
      }
    }
  }
  feature {
    key: "dropoff_location"
    value {
      bytes_list {
        value: "POINT (-87.633308 41.899602)"
      }
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
      float_list {
        value: -87.63330841064453
      }
    }
  }
  feature {
    key: "extras"
    value {
      float_list {
        value: 1.5
      }
    }
  }
  feature {
    key: "fare"
    value {
      float_list {
        value: 12.75
      }
    }
  }
  feature {
    key: "payment_type"
    value {
      b

In [7]:


b64_example = base64.b64encode(serialized_example).decode("utf-8")
instances_packet = [{
  "examples": {
    "b64": b64_example
  }
}]
aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location='us-central1')
endpoint = aiplatform.Endpoint(ENDPOINT)
prediction = endpoint.predict(instances=instances_packet)

# Extract and print only the prediction value
predicted_value = prediction.predictions[0][0]
print(f"Prediction output: {predicted_value}")


Prediction output: 15.501709
